In [8]:
# Directories
import os

new_directory = r'c://Users//Fer//TESIS_ARCHIVOS//TESIS_AIRE//MP_Forecasting//aqi_forecasting//notebooks'
os.chdir(new_directory)

# Data Manipulation
import pandas as pd # for data manipulation
import numpy as np # for data manipulation


# Training utils
from training_code.utils import utils_xgboost

# Optuna
import optuna
import joblib
import pickle

# Tiempo
import datetime as dt
from dateutil.relativedelta import relativedelta, MO

# Modelos
from sklearn.preprocessing import MinMaxScaler
from sklearn.multioutput import MultiOutputRegressor
import xgboost as xgb


# Metricas

# Visualizations
import plotly.graph_objects as go # for data visualization
import plotly.express as px # for data visualization
import matplotlib.pyplot as plt

# Advertencias
import warnings
warnings.filterwarnings("ignore")

In [9]:
datos = pd.read_csv('datos/230127_train_ESTACIONES.csv', parse_dates=['FECHAHORA'])
validacion = pd.read_csv('datos/230127_test_ESTACIONES.csv', parse_dates=['FECHAHORA'])

lista = [datos,validacion]

df = pd.concat(lista)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1580842 entries, 0 to 264959
Data columns (total 28 columns):
 #   Column                  Non-Null Count    Dtype         
---  ------                  --------------    -----         
 0   ESTACION                1580842 non-null  int64         
 1   FECHAHORA               1580842 non-null  datetime64[ns]
 2   ANHO                    1580842 non-null  int64         
 3   DIA                     1580842 non-null  int64         
 4   MES                     1580842 non-null  int64         
 5   HORA                    1580842 non-null  int64         
 6   MINUTO                  1580842 non-null  int64         
 7   MP1                     1580842 non-null  float64       
 8   MP2_5                   1580842 non-null  float64       
 9   MP10                    1580842 non-null  float64       
 10  TEMPERATURA             1580842 non-null  float64       
 11  HUMEDAD                 1580842 non-null  float64       
 12  PRESION        

# 6 horas

In [11]:
resample_item = 5
horizon_item = 0.25
previous_item = 2

for station_item in range(1,11):

    estacion = station_item

    variables = ['MP1', 'MP2_5', 'MP10', 'AQI_MP10', 'AQI_MP2_5']

    dependent = ['AQI_MP2_5']

    number_of_features = len(variables)

    training_days = previous_item
    forecast_days = horizon_item

    if resample_item == 5:
        samples_per_day = 288
    elif resample_item == 30:
        samples_per_day = 48
    elif resample_item == 60:
        samples_per_day = 24

    step = forecast_days*samples_per_day

    train_months = relativedelta(months = 15)

    input_samples = int(samples_per_day * training_days) # cantidad de muestras previas
    output_samples = int(samples_per_day * forecast_days) # cantidad de muestras horizonte
    train_test_samples = int(input_samples + output_samples) # cantidad de datos para el train_test

    X_train, y_train, X_test, y_test = utils_xgboost.get_everything(df, 
                                                        estacion,
                                                        train_months, 
                                                        variables, 
                                                        dependent, 
                                                        train_test_samples, 
                                                        input_samples, 
                                                        output_samples, 
                                                        number_of_features,
                                                        step)


    study = joblib.load("optuna_studies/XGBOOST/experimento_full/Validaciones/estacion_"+str(station_item)+"_study_XGBOOST_mejoresparam40_resample_"+str(resample_item)+"_horizon_"+str(horizon_item)+"_previous_"+str(previous_item)+".pkl")

    params = study.best_params

                # study = joblib.load("optuna_studies/XGBOOST/experimento_full/study_XGBOOST_resample_"+str(resample_item)+"_horizon_"+str(horizon_item)+"_previous_"+str(previous_item)+".pkl")

    xgb_model = xgb.XGBRegressor(** params)

    trained_xgb_model = MultiOutputRegressor(xgb_model).fit(X_train , y_train)

    joblib.dump(trained_xgb_model, "final_models/station_" + str(station_item) + "_6hs.pkl")

    print('hecho!')

hecho!
hecho!
hecho!
hecho!
hecho!
hecho!
hecho!
hecho!
hecho!
hecho!


# 24 horas

In [14]:
resample_item = 30
horizon_item = 1
previous_item = 2

for station_item in range(1,11):

    estacion = station_item

    variables = ['MP1', 'MP2_5', 'MP10', 'AQI_MP10', 'AQI_MP2_5']

    dependent = ['AQI_MP2_5']

    number_of_features = len(variables)

    training_days = previous_item
    forecast_days = horizon_item

    if resample_item == 5:
        samples_per_day = 288
    elif resample_item == 30:
        samples_per_day = 48
    elif resample_item == 60:
        samples_per_day = 24

    step = forecast_days*samples_per_day

    train_months = relativedelta(months = 15)

    input_samples = int(samples_per_day * training_days) # cantidad de muestras previas
    output_samples = int(samples_per_day * forecast_days) # cantidad de muestras horizonte
    train_test_samples = int(input_samples + output_samples) # cantidad de datos para el train_test

    X_train, y_train, X_test, y_test = utils_xgboost.get_everything(df, 
                                                        estacion,
                                                        train_months, 
                                                        variables, 
                                                        dependent, 
                                                        train_test_samples, 
                                                        input_samples, 
                                                        output_samples, 
                                                        number_of_features,
                                                        step)


    study = joblib.load("optuna_studies/XGBOOST/experimento_full/Validaciones/estacion_"+str(station_item)+"_study_XGBOOST_mejoresparam40_resample_"+str(resample_item)+"_horizon_"+str(horizon_item)+"_previous_"+str(previous_item)+".pkl")

    params = study.best_params

                # study = joblib.load("optuna_studies/XGBOOST/experimento_full/study_XGBOOST_resample_"+str(resample_item)+"_horizon_"+str(horizon_item)+"_previous_"+str(previous_item)+".pkl")

    xgb_model = xgb.XGBRegressor(** params)

    trained_xgb_model = MultiOutputRegressor(xgb_model).fit(X_train , y_train)

    joblib.dump(trained_xgb_model, "final_models/station_" + str(station_item) + "_24hs.pkl")

    print('hecho!')

hecho!
hecho!
hecho!
hecho!
hecho!
hecho!
hecho!
hecho!
hecho!
hecho!
